# Экстрактивная суммаризация с BERT

## Описание задачи

Необходимо провести суммаризацию текста, применив методы экстрактивной суммаризации.

Экстрактивная суммаризация – это метод автоматического извлечения ключевых предложений из исходного текста для формирования краткого содержания. В отличие от абстрактивной суммаризации, где создаются новые формулировки, экстрактивный подход берет существующие фразы и упорядочивает их в логичной последовательности.

Что нужно сделать?
Использовать BERT-модели – например, KeyBERT для выделения ключевых фраз, https://human.spbstu.ru/userfiles/files/articles/2024/1/20-35.pdf

Результат:
* код .py, .ipynb
* выводы


## Импорты и настройки

In [1]:
!pip install -U pandas

In [2]:
!pip install keybert

In [3]:
!pip install transformers -U

  Using cached transformers-4.51.3-py3-none-any.whl.metadata (38 kB)
  Using cached huggingface_hub-0.30.2-py3-none-any.whl.metadata (13 kB)
Using cached transformers-4.51.3-py3-none-any.whl (10.4 MB)
Using cached huggingface_hub-0.30.2-py3-none-any.whl (481 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface_hub 0.24.5
    Uninstalling huggingface_hub-0.24.5:
      Successfully uninstalled huggingface_hub-0.24.5
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3


In [4]:
import nltk

import pandas as pd
import matplotlib.pyplot as plt

from keybert import KeyBERT
from nltk.corpus import stopwords

C:\Users\cake\anaconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [5]:
# Отображение до 50 столбцов в таблицах
pd.set_option('display.max_columns', 50)

# Сброс кастомного отображения столбцов в таблицах
# pd.reset_option('display.max_columns')

## Обзор данных 

Для данной задачи будем использовать уже распарсенный JSON чата, который был сохранён в CSV-формате

In [6]:
df = pd.read_csv("../def_Klyusnik_A/output/processed_messages_1.csv")

In [7]:
df.head(25)

,name,type,id_first,id,type.1,date,date_unixtime,actor,actor_id,action,inviter,text,text_entities,from,from_id,reply_to_message_id,edited,edited_unixtime,reactions,file,file_name,file_size,thumbnail,thumbnail_file_size,media_type,sticker_emoji,mime_type,duration_seconds,width,height,forwarded_from,saved_from,photo,photo_file_size,via_bot,members
0,💬 Data Practicum Chat,private_supergroup,1379846874,266690,service,2025-02-01T22:40:19,1738438819,Елизавета,user200103497,join_group_by_link,Group,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,💬 Data Practicum Chat,private_supergroup,1379846874,266691,service,2025-02-02T21:59:34,1738522774,Agamet Agametov,user217302209,join_group_by_link,Group,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,💬 Data Practicum Chat,private_supergroup,1379846874,266693,message,2025-02-03T11:28:38,1738571318,NaN,NaN,NaN,NaN,['Всем большой привет! Приглашаю на свой уютны...,"[{'type': 'plain', 'text': 'Всем большой приве...",Olga Varavina,user312724902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,💬 Data Practicum Chat,private_supergroup,1379846874,266694,message,2025-02-03T11:52:20,1738572740,NaN,NaN,NaN,NaN,А у тебя когда будет свой канал про аналитику?,"[{'type': 'plain', 'text': 'А у тебя когда буд...",Илья,user1349934990,266689.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,💬 Data Practicum Chat,private_supergroup,1379846874,266695,message,2025-02-03T11:52:37,1738572757,NaN,NaN,NaN,NaN,Будешь туда голосовухи пятиминутные постить,"[{'type': 'plain', 'text': 'Будешь туда голосо...",Илья,user1349934990,NaN,2025-02-03T11:58:48,1.738573e+09,"[{'type': 'emoji', 'count': 4, 'emoji': '😁', '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,💬 Data Practicum Chat,private_supergroup,1379846874,266696,message,2025-02-03T11:55:09,1738572909,NaN,NaN,NaN,NaN,"Потому что сделаны так, будто устарели уже лет...","[{'type': 'plain', 'text': 'Потому что сделаны...",Sergey,user60031833,266654.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,💬 Data Practicum Chat,private_supergroup,1379846874,266697,message,2025-02-03T11:56:57,1738573017,NaN,NaN,NaN,NaN,Подкаст?),"[{'type': 'plain', 'text': 'Подкаст?)'}]",Sergey,user60031833,266695.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,💬 Data Practicum Chat,private_supergroup,1379846874,266698,message,2025-02-03T11:57:17,1738573037,NaN,NaN,NaN,NaN,"Не, это не так раздражает","[{'type': 'plain', 'text': 'Не, это не так раз...",Илья,user1349934990,266697.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,💬 Data Practicum Chat,private_supergroup,1379846874,266699,message,2025-02-03T11:57:36,1738573056,NaN,NaN,NaN,NaN,"Нужны голосовуки с эээ, нууу, и 10 секундными ...","[{'type': 'plain', 'text': 'Нужны голосовуки с...",Илья,user1349934990,NaN,2025-02-03T11:58:05,1.738573e+09,"[{'type': 'emoji', 'count': 1, 'emoji': '😁', '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,💬 Data Practicum Chat,private_supergroup,1379846874,266700,message,2025-02-03T11:58:06,1738573086,NaN,NaN,NaN,NaN,"Звонили из ада, просили передать что ждут на м...","[{'type': 'plain', 'text': 'Звонили из ада, пр...",Aleksey Voronin,user234467651,266699.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1007 entries, 0 to 1006
Data columns (total 36 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   name                 1007 non-null   object 
 1   type                 1007 non-null   object 
 2   id_first             1007 non-null   int64  
 3   id                   1007 non-null   int64  
 4   type.1               1007 non-null   object 
 5   date                 1007 non-null   object 
 6   date_unixtime        1007 non-null   int64  
 7   actor                83 non-null     object 
 8   actor_id             83 non-null     object 
 9   action               83 non-null     object 
 10  inviter              81 non-null     object 
 11  text                 881 non-null    object 
 12  text_entities        1007 non-null   object 
 13  from                 924 non-null    object 
 14  from_id              924 non-null    object 
 15  reply_to_message_id  689 non-null    f

In [9]:
df.text.head(15)

0                                                   NaN
1                                                   NaN
2     ['Всем большой привет! Приглашаю на свой уютны...
3        А у тебя когда будет свой канал про аналитику?
4           Будешь туда голосовухи пятиминутные постить
5     Потому что сделаны так, будто устарели уже лет...
6                                             Подкаст?)
7                             Не, это не так раздражает
8     Нужны голосовуки с эээ, нууу, и 10 секундными ...
9     Звонили из ада, просили передать что ждут на м...
10                Ты хотел сказать "ждут мастер-класс"?
11    Их сделали, чтобы понравится человеку, котором...
12                      Благодарствую, ещё не проснулся
13    Боже теперь и я в этом рейтинге😆😆😆 и тебе спасибо
14                                          зря конечно
Name: text, dtype: object

In [10]:
df.text_entities.head(15)

0                                                    []
1                                                    []
2     [{'type': 'plain', 'text': 'Всем большой приве...
3     [{'type': 'plain', 'text': 'А у тебя когда буд...
4     [{'type': 'plain', 'text': 'Будешь туда голосо...
5     [{'type': 'plain', 'text': 'Потому что сделаны...
6              [{'type': 'plain', 'text': 'Подкаст?)'}]
7     [{'type': 'plain', 'text': 'Не, это не так раз...
8     [{'type': 'plain', 'text': 'Нужны голосовуки с...
9     [{'type': 'plain', 'text': 'Звонили из ада, пр...
10    [{'type': 'plain', 'text': 'Ты хотел сказать "...
11    [{'type': 'plain', 'text': 'Их сделали, чтобы ...
12    [{'type': 'plain', 'text': 'Благодарствую, ещё...
13    [{'type': 'plain', 'text': 'Боже теперь и я в ...
14           [{'type': 'plain', 'text': 'зря конечно'}]
Name: text_entities, dtype: object

### Вывод

В нашем распоряжении имеется выгрузка чата, которая содержит 1007 строк и 36 столбцов. Имеется множество пропусков в данных.

Ключевые признаки для нас: `text` и `text_entities`

## EDA и предобработка данных

### Проверка уникальных значений

Проверим некоторые признаки на уникальные значения.

In [11]:
df.name.unique()

array(['💬 Data Practicum Chat'], dtype=object)

In [12]:
df.type.unique()

array(['private_supergroup'], dtype=object)

In [13]:
df.id_first.unique()

array([1379846874], dtype=int64)

In [14]:
df.action.unique()

array(['join_group_by_link', nan, 'invite_members'], dtype=object)

In [15]:
df.file.unique()

array([nan,
       '(File not included. Change data exporting settings to download.)'],
      dtype=object)

In [16]:
df.media_type.unique()

array([nan, 'sticker', 'animation', 'video_file'], dtype=object)

In [17]:
df.members.unique()

array([nan, "['Руслан']", "['Oleg Zhukov']"], dtype=object)

Признаки `name`, `type`, `id_first` свидетельствуют, что мы имеем дело с выгрузкой из одного чата. Множество признаков выполняют техническую роль: например показывают размер загруженного файла, тип медиа, дату редактирования сообщения.

### Обработка дат

Мы будем пытаться объединить сообщения в текст по дням. Для этого нам нужно привести даты к правильному формату.

In [18]:
df.date = pd.to_datetime(df.date)
df.date

0      2025-02-01 22:40:19
1      2025-02-02 21:59:34
2      2025-02-03 11:28:38
3      2025-02-03 11:52:20
4      2025-02-03 11:52:37
               ...        
1002   2025-02-27 14:20:29
1003   2025-02-27 18:27:05
1004   2025-02-27 18:33:30
1005   2025-02-27 20:01:03
1006   2025-02-27 22:33:36
Name: date, Length: 1007, dtype: datetime64[ns]

### Изменение индекса

Заменим индекс датасета признаком `date` для проверки на монотонность.

In [19]:
df = df.set_index('date')
df.head()

,name,type,id_first,id,type.1,date_unixtime,actor,actor_id,action,inviter,text,text_entities,from,from_id,reply_to_message_id,edited,edited_unixtime,reactions,file,file_name,file_size,thumbnail,thumbnail_file_size,media_type,sticker_emoji,mime_type,duration_seconds,width,height,forwarded_from,saved_from,photo,photo_file_size,via_bot,members
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2025-02-01 22:40:19,💬 Data Practicum Chat,private_supergroup,1379846874,266690,service,1738438819,Елизавета,user200103497,join_group_by_link,Group,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-02-02 21:59:34,💬 Data Practicum Chat,private_supergroup,1379846874,266691,service,1738522774,Agamet Agametov,user217302209,join_group_by_link,Group,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-02-03 11:28:38,💬 Data Practicum Chat,private_supergroup,1379846874,266693,message,1738571318,NaN,NaN,NaN,NaN,['Всем большой привет! Приглашаю на свой уютны...,"[{'type': 'plain', 'text': 'Всем большой приве...",Olga Varavina,user312724902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-02-03 11:52:20,💬 Data Practicum Chat,private_supergroup,1379846874,266694,message,1738572740,NaN,NaN,NaN,NaN,А у тебя когда будет свой канал про аналитику?,"[{'type': 'plain', 'text': 'А у тебя когда буд...",Илья,user1349934990,266689.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-02-03 11:52:37,💬 Data Practicum Chat,private_supergroup,1379846874,266695,message,1738572757,NaN,NaN,NaN,NaN,Будешь туда голосовухи пятиминутные постить,"[{'type': 'plain', 'text': 'Будешь туда голосо...",Илья,user1349934990,NaN,2025-02-03T11:58:48,1.738573e+09,"[{'type': 'emoji', 'count': 4, 'emoji': '😁', '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Проверка индекса на монотонность

Проверим, что новый индекс монотонно возрастает.

In [20]:
df = df.sort_index()
df.index.is_monotonic_increasing

True

In [21]:
df.index

DatetimeIndex(['2025-02-01 22:40:19', '2025-02-02 21:59:34',
               '2025-02-03 11:28:38', '2025-02-03 11:52:20',
               '2025-02-03 11:52:37', '2025-02-03 11:55:09',
               '2025-02-03 11:56:57', '2025-02-03 11:57:17',
               '2025-02-03 11:57:36', '2025-02-03 11:58:06',
               ...
               '2025-02-27 13:27:37', '2025-02-27 13:29:02',
               '2025-02-27 14:08:46', '2025-02-27 14:10:05',
               '2025-02-27 14:11:43', '2025-02-27 14:20:29',
               '2025-02-27 18:27:05', '2025-02-27 18:33:30',
               '2025-02-27 20:01:03', '2025-02-27 22:33:36'],
              dtype='datetime64[ns]', name='date', length=1007, freq=None)

Теперь мы можем быть уверены в том, что в нашем распоряжении только данные с 01.02.2025 по 27.02.2025 включительно.

### Выделение дня месяца

В отдельный признак выделим день месяца. В дальнейшем мы будем группировать сообщения по нему.

In [22]:
df['day_of_month'] = df.index.day
df.head()

,name,type,id_first,id,type.1,date_unixtime,actor,actor_id,action,inviter,text,text_entities,from,from_id,reply_to_message_id,edited,edited_unixtime,reactions,file,file_name,file_size,thumbnail,thumbnail_file_size,media_type,sticker_emoji,mime_type,duration_seconds,width,height,forwarded_from,saved_from,photo,photo_file_size,via_bot,members,day_of_month
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2025-02-01 22:40:19,💬 Data Practicum Chat,private_supergroup,1379846874,266690,service,1738438819,Елизавета,user200103497,join_group_by_link,Group,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2025-02-02 21:59:34,💬 Data Practicum Chat,private_supergroup,1379846874,266691,service,1738522774,Agamet Agametov,user217302209,join_group_by_link,Group,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
2025-02-03 11:28:38,💬 Data Practicum Chat,private_supergroup,1379846874,266693,message,1738571318,NaN,NaN,NaN,NaN,['Всем большой привет! Приглашаю на свой уютны...,"[{'type': 'plain', 'text': 'Всем большой приве...",Olga Varavina,user312724902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
2025-02-03 11:52:20,💬 Data Practicum Chat,private_supergroup,1379846874,266694,message,1738572740,NaN,NaN,NaN,NaN,А у тебя когда будет свой канал про аналитику?,"[{'type': 'plain', 'text': 'А у тебя когда буд...",Илья,user1349934990,266689.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
2025-02-03 11:52:37,💬 Data Practicum Chat,private_supergroup,1379846874,266695,message,1738572757,NaN,NaN,NaN,NaN,Будешь туда голосовухи пятиминутные постить,"[{'type': 'plain', 'text': 'Будешь туда голосо...",Илья,user1349934990,NaN,2025-02-03T11:58:48,1.738573e+09,"[{'type': 'emoji', 'count': 4, 'emoji': '😁', '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3


In [23]:
# Проверим уникальные значения в новом признаке
df.day_of_month.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27])

В нашем датасете есть сообщения за каждый день с 1 по 27 февраля включительно.

### Группировка сообщений по дням

In [24]:
# Удаляем строки, где text является NaN
cleaned_df = df.dropna(subset=['text'])

In [25]:
# Группируем по day_of_month и объединяем тексты
grouped_messages = cleaned_df.groupby('day_of_month')['text'].apply(lambda x: ' '.join(x)).reset_index()

In [26]:
grouped_messages

,day_of_month,text
0,3,['Всем большой привет! Приглашаю на свой уютны...
1,4,"[{'type': 'bold', 'text': 'Предложение поучаст..."
2,5,форма работает! Форма открыта на сбор заявок д...
3,6,Привет всем ) А кто-то из GA4 выгружал сырые д...
4,7,Всем привет! 12.10.2024 я закончил курс Аналит...
5,8,"[{'type': 'bot_command', 'text': '/toprep@yndx..."
6,9,"Всем привет! Нужна помощь, есть задание много-..."
7,10,"[{'type': 'bot_command', 'text': '/toprep@yndx..."
8,11,['Всем привет!\n\nНа связи снова команда иссле...
9,12,думаешь люди до 200 живут 🤔 Она с Кавказа [{'t...


## Экстрактивная суммаризация

### V.0 

In [27]:
# загрузка списка стоп-слов
nltk.download('stopwords')
russian_stopwords = stopwords.words('russian')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cake\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [28]:
# Загрузка модели (используется 'paraphrase-multilingual-MiniLM-L12-v2' для русского)
kw_model = KeyBERT(model='paraphrase-multilingual-MiniLM-L12-v2')

def summarize_text(text, n_phrases=3, stop_words=russian_stopwords):
    # Извлекаем ключевые фразы
    keywords = kw_model.extract_keywords(
        text,
        keyphrase_ngram_range=(1, 2),  # Размер фраз
        stop_words=stop_words,         # Стоп-слова
        top_n=n_phrases,
        diversity=0.5                  # Разнообразие результатов
    )
    return [phrase for phrase, _ in keywords]

# Применяем к каждому тексту
grouped_messages['summary'] = grouped_messages['text'].apply(summarize_text)

grouped_messages[['day_of_month', 'summary']]

C:\Users\cake\anaconda3\Lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


,day_of_month,summary
0,3,"[канал аналитику, webm будешь, аналитику будешь]"
1,4,"[аналитика type, курсов аналитики, курсов анал..."
2,5,"[оставить заявку, заявок завтра, отправил заявку]"
3,6,"[ga4 выгружал, всем ga4, неактивна заявку]"
4,7,"[сколько времени, закончил курс, резюме сопров..."
5,8,"[type bot_command, bot_command, bot_command text]"
6,9,"[классовой классификации, классификации, класс..."
7,10,"[type bot_command, bot_command, сяпас бот]"
8,11,"[дальнейшем обучении, регулярное обучение, обу..."
9,12,"[кавказа, хотели пожалуйста, окей кофе]"


### V.0.1

По первым результатам можно выделить дополнительные стоп-слова

In [29]:
# Добавляем несколько стоп-слов после первой итерации
custom_stopwords = ['type', 'bot_command', 'bold', 'text', 'custom_emoji']
all_stopwords = list(set(russian_stopwords + custom_stopwords))

In [30]:
# Применяем к каждому тексту нашу функцию и новый список стоп-слов
grouped_messages['summary'] = grouped_messages['text'].apply(lambda x: summarize_text(x, stop_words=all_stopwords))
grouped_messages[['day_of_month', 'summary']]

,day_of_month,summary
0,3,"[канал аналитику, webm будешь, аналитику будешь]"
1,4,"[курсов аналитики, курсов аналитик, аналитики ..."
2,5,"[оставить заявку, заявок завтра, отправил заявку]"
3,6,"[ga4 выгружал, всем ga4, неактивна заявку]"
4,7,"[сколько времени, закончил курс, резюме сопров..."
5,8,"[yndxcbot чот, yndxcbot toprep, toprep yndxcbot]"
6,9,"[классовой классификации, классификации, класс..."
7,10,"[сяпас бот, бот, бот такое]"
8,11,"[дальнейшем обучении, регулярное обучение, обу..."
9,12,"[кавказа toprep, кавказа, хотели пожалуйста]"


### Вывод

Сделана базовая реализация экстрактивной суммаризации с BERT. По результатам первой итерации добавлено несколько новых стоп-слов. Полученный результат далёк от оптимального.